# Jetson Inference ONNX Demo (JetRacer)

This notebook runs a custom ONNX SSD-style detector with **jetson.inference** and
shows results in the notebook. It is similar to `onnx_detect_demo.ipynb` but uses
Jetson Inference instead of ONNX Runtime.

Notes:
- You need `jetson-inference`, `jetson-utils`, and `jetcam` installed on the Jetson.
- Set the correct input/output blob names for your ONNX (see the config cell).
- Typical SSD exports use `input_0`, `scores`, and `boxes`.


In [ ]:
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt

import jetson.inference
import jetson.utils


In [ ]:
# ---- Config ----
repo_root = Path.cwd()

model_path = repo_root / 'mobilenetv2_ssd.onnx'  # change to your ONNX path
labels_path = repo_root / 'finetune_dataset' / 'classes.txt'

# These must match the ONNX graph
input_blob = 'input_0'
output_cvg = 'scores'
output_bbox = 'boxes'

threshold = 0.35
overlay = 'box,labels,conf'


In [ ]:
# ---- Load network ----
argv = [
    f'--model={model_path}',
    f'--labels={labels_path}',
    f'--input-blob={input_blob}',
    f'--output-cvg={output_cvg}',
    f'--output-bbox={output_bbox}',
    f'--threshold={threshold}',
]

net = jetson.inference.detectNet(argv=argv)
print('Loaded network:', net)


In [ ]:
def show_bgr(image_bgr, title=None):
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    if title:
        plt.title(title)
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()


## Single image


In [ ]:
image_path = repo_root / 'test_images' / 'collect1_frame_000861.jpg'
image = cv2.imread(str(image_path))
if image is None:
    raise FileNotFoundError(image_path)

# OpenCV BGR -> RGBA for jetson.utils
image_rgba = cv2.cvtColor(image, cv2.COLOR_BGR2RGBA)
cuda_img = jetson.utils.cudaFromNumpy(image_rgba)

detections = net.Detect(cuda_img, overlay=overlay)
print(f'Detections: {len(detections)}')

# Convert back to numpy (RGBA -> BGR) for display
out_rgba = jetson.utils.cudaToNumpy(cuda_img)
out_bgr = cv2.cvtColor(out_rgba, cv2.COLOR_RGBA2BGR)
show_bgr(out_bgr, title='Detections')


## Folder batch (timing + FPS)


In [ ]:
import time
from pathlib import Path

images_dir = repo_root / 'test_images'  # change to your folder
image_paths = sorted([p for p in images_dir.iterdir() if p.suffix.lower() in {'.jpg', '.png', '.jpeg', '.bmp'}])
if not image_paths:
    raise FileNotFoundError(f'No images found in {images_dir}')

total = 0
total_time = 0.0
for image_path in image_paths:
    image = cv2.imread(str(image_path))
    if image is None:
        continue
    image_rgba = cv2.cvtColor(image, cv2.COLOR_BGR2RGBA)
    cuda_img = jetson.utils.cudaFromNumpy(image_rgba)

    start = time.perf_counter()
    detections = net.Detect(cuda_img, overlay=overlay)
    elapsed = time.perf_counter() - start

    total += 1
    total_time += elapsed

if total == 0:
    raise RuntimeError('No readable images in folder')

avg_ms = (total_time / total) * 1000.0
fps = total / total_time if total_time > 0 else float('inf')
print(f'Processed {total} images')
print(f'Avg time: {avg_ms:.2f} ms | FPS: {fps:.2f}')


## JetRacer camera (CSI)
This uses `jetcam` to read frames and displays results in the notebook.


In [ ]:
from IPython.display import display, clear_output
from jetcam.csi_camera import CSICamera
import gc

camera = None

def iter_camera_frames(num_frames=50, width=640, height=480, fps=30):
    global camera
    camera = CSICamera(width=width, height=height, fps=fps)
    for _ in range(num_frames):
        frame = camera.read()
        if frame is None:
            continue
        yield frame

def stop_camera():
    global camera
    if camera is None:
        return
    if hasattr(camera, 'stop'):
        camera.stop()
    if hasattr(camera, 'release'):
        camera.release()
    camera = None
    gc.collect()

for frame in iter_camera_frames(num_frames=50):
    # frame is BGR
    frame_rgba = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    cuda_img = jetson.utils.cudaFromNumpy(frame_rgba)
    detections = net.Detect(cuda_img, overlay=overlay)

    out_rgba = jetson.utils.cudaToNumpy(cuda_img)
    out_bgr = cv2.cvtColor(out_rgba, cv2.COLOR_RGBA2BGR)

    clear_output(wait=True)
    plt.figure(figsize=(6, 4))
    plt.imshow(out_bgr[:, :, ::-1])
    plt.axis('off')
    display(plt.gcf())
    plt.close()

# stop camera when loop ends
stop_camera()
